# Initial installs

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes --upgrade --quiet

In [ ]:
!pip install accelerate --upgrade --quiet

In [ ]:
!pip install PyMuPDF --quiet

In [ ]:
!pip install pyngrok --quiet

# Imports

In [ ]:
import bitsandbytes
import accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, BertTokenizer, BertForSequenceClassification

In [ ]:
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
import requests
import json
import math
import gc
import os
import re

In [ ]:
from huggingface_hub import notebook_login
from google.colab import drive, userdata

In [ ]:
import llama_index
import llama_index.readers
import llama_index.readers.file
from llama_index.readers.file import PyMuPDFReader
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, download_loader
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

from pathlib import Path

# HuggingFace/Drive Interfacing

In [ ]:
notebook_login()

In [ ]:
drive.mount('/content/drive')

# GPU

In [ ]:
# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Free GPU Memory

In [ ]:
def free_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()

# Load in the Model

In [ ]:
# Load tokenizer and final trained model
model_directory = "/content/drive/MyDrive/saved_models/LLama2-7B-chat-PT1-v2"
auth_token = userdata.get('HF_TOKEN')

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_directory,
    local_files_only = True
)

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the fine-tuned model
model = AutoModelForCausalLM.from_pretrained(
    model_directory,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    rope_scaling={"type": "dynamic", "factor": 2},
    local_files_only = True,
    quantization_config=quantization_config
)

In [ ]:
model.eval()
print("Model loaded successfully.")

# Load Initial Classifier

In [ ]:
classifier_tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
stage1_classifier = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/classifiers/v1')

In [ ]:
stage1_classifier.to(device)
stage1_classifier.eval()
print("Classifer loaded successfully.")

In [ ]:
# Tokenization and input formatting
def encode_data(tokenizer, texts, max_len=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = max_len,
            truncation = True,
            padding = 'max_length',
            return_attention_mask = True,
            return_tensors = 'pt',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)

    return input_ids, attention_masks

In [ ]:
def classify(model, tokenizer, text, label_dict):
    # Encode the text using the provided tokenizer
    input_ids, attention_masks = encode_data(tokenizer, [text])

    # Move tensors to the same device as the model
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    # Predict
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids = None, attention_mask = attention_masks)

    # Extract the predicted index and map to label name
    prediction_idx = torch.argmax(outputs.logits, dim = 1).item()
    return label_dict[prediction_idx]

In [ ]:
label_dict = {0: 'quantitative analysis', 1: 'general information', 2: 'miscellaneous'}

# Webpage Layout Interface

In [ ]:
STATIC_DIR = os.path.abspath('/content/interface/static')

# Branches

## Knowledge Branch

In [ ]:
knowledge_system_prompt = """[INST] <<SYS>>
You are an informative assistant called "Assistant". Your goal is to provide accurate and relevant information about Cardiovascular disease and adjacent topics in response to the user's queries.
Please ensure that your responses are informative, helpful, direct, dispassionate, and factual. Respond in plain English, and aim for your response to be at least 3 sentences in length.
Do not include introductory phrases such as: "Sure! I can help with that", "Of course", "I believe", "Based on the context", or any other form of greeting.
Do not include summary statements such as "I hope this helps," "Let me know if you need anything else," "In summary," or "To conclude."
If you're uncertain about a question, it's better to admit it rather than provide inaccurate information.
Directly answer to the User's question ONLY, without preamble or reference to the provided context. Do not impersonate the User and do not include followup questions in your response unless prompted.
Do NOT reference the provided context or provided information in your answer as the User is aware of these already.
REPEAT: Do NOT reference the context or include phrases like "based on the context" in your response.
<</SYS>>
"""

query_wrapper_prompt = SimpleInputPrompt("{query_str}\nAssistant: [/INST]")

In [ ]:
# Create a HF LLM using the llama index wrapper
knowledge_llm = HuggingFaceLLM(
    context_window = 4096,
    max_new_tokens = 512,
    generate_kwargs = {"temperature": 0.6},
    system_prompt = knowledge_system_prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    model = model,
    tokenizer = tokenizer
)

In [ ]:
# Create and dl embeddings instance
embeddings = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
)

In [ ]:
# Function to load and index multiple PDF documents
def load_and_index_documents(directory_path):
    loader = PyMuPDFReader()
    all_documents = []
    for pdf_file in Path(directory_path).rglob('*.pdf'):
        documents = loader.load(file_path = pdf_file, metadata = True)
        all_documents.extend(documents)

    # Create an index with all documents
    index = VectorStoreIndex.from_documents(
        all_documents,
        embed_model = embeddings
    )

    return index

In [ ]:
# Load and index documents from a specified directory
directory_path = '/content/drive/MyDrive/data/'
doc_index = load_and_index_documents(directory_path)

def serve_knowledge(prompt):
    # Setup index query engine using LLM
    query_engine = doc_index.as_query_engine(llm = knowledge_llm)

    response = query_engine.query(prompt)
    return {"text": response.response.strip(), "line": None}

## Quantitative Branch

### Quantitative Calculate Functions

In [ ]:
# average LDL values for men and women from ages 30 to 80
men_mean_LDL = [3.3174731188868307, 3.334143838077217, 3.350898329725846, 3.3677370147998453, 3.3846603163817544, 3.4016686596801553, 3.4187624720403567, 3.435942182955132, 3.45320822407551, 3.4705610292216185, 3.4880010343935868, 3.4955003878975948, 3.54100077579519, 3.555500387897595, 3.611998965606413, 3.655001292991983, 3.6629997414016033, 3.6299999999999994, 3.6829997414016034, 3.6814998707008013, 3.695500387897595, 3.68899922420481, 3.700499094905611, 3.724998707008017, 3.7459994828032066, 3.7070002585983968, 3.7089992242048098, 3.714000517196793, 3.721499870700802, 3.714000517196793, 3.718500129299198, 3.69899922420481, 3.6940005171967933, 3.6870002585983968, 3.6940005171967933, 3.6570002585983965, 3.6470002585983967, 3.6449987070080163, 3.631998965606413, 3.613838970778381, 3.595769775924489, 3.5777909270448665, 3.559901972409642, 3.542102462547594, 3.524391950234856, 3.506769990483682, 3.4892361405312635, 3.471789959828607, 3.454431010029464, 3.437158854979317, 3.41997306070442]
women_mean_LDL = [2.861743860038676, 2.8833691285024443, 2.905157812093143, 2.9271111456857857, 2.949230373486938, 2.971516749105227, 2.9939715356223946, 3.016596005664881, 3.03939144147595, 3.062359134988363, 3.085500387897595, 3.0970002585983964, 3.12, 3.162999741401603, 3.1659994828032065, 3.2199999999999998, 3.2514998707008016, 3.3110007757951894, 3.351998965606413, 3.43, 3.497000258598397, 3.5314998707008014, 3.601998965606413, 3.654000517196793, 3.7199999999999998, 3.7695009050943886, 3.7919989656064126, 3.821998965606413, 3.8435014222911814, 3.83899922420481, 3.9010007757951897, 3.9270002585983965, 3.934998707008017, 3.9259994828032063, 3.9440005171967933, 3.9495009050943883, 3.9449987070080166, 3.9380010343935865, 3.9570002585983968, 3.9329997414016034, 3.9529997414016034, 3.9332347426945957, 3.9135685689811224, 3.8940007261362166, 3.8745307225055354, 3.8551580688930076, 3.8358822785485422, 3.8167028671558, 3.797619352820021, 3.7786312560559208, 3.759738099775641]

# average SBP values for men and women from ages 30 to 80
men_mean_SBP = [127.09770000000009, 127.64270000000009, 128.1877000000001, 128.73270000000008, 129.27770000000007, 129.82270000000005, 130.36770000000004, 130.91270000000003, 131.45770000000002, 132.0027, 132.5477, 132.5595, 133.0417, 133.9244, 133.8602, 134.1759, 134.8107, 135.0133, 136.0058, 136.2948, 136.8521, 137.2601, 137.7067, 138.7961, 139.1646, 139.5142, 140.0917, 140.7533, 141.3936, 142.006, 142.6065, 143.1211, 143.6272, 144.0828, 145.4364, 145.5392, 146.494, 146.6693, 146.9347, 147.3451, 147.8901, 148.43509999999998, 148.98009999999996, 149.52509999999995, 150.07009999999994, 150.61509999999993, 151.16009999999991, 151.7050999999999, 152.2500999999999, 152.79509999999988, 153.34009999999986]
women_mean_SBP = [111.82199999999999, 112.72099999999999, 113.61999999999999, 114.51899999999999, 115.41799999999999, 116.317, 117.216, 118.115, 119.014, 119.913, 120.812, 121.5938, 122.2175, 122.8461, 123.4768, 124.808, 125.45, 126.2387, 127.1368, 128.1205, 129.2104, 130.3305, 131.2608, 132.1755, 132.5652, 133.3094, 134.2712, 135.0678, 136.1804, 137.2556, 137.6882, 139.0932, 140.3054, 141.3204, 142.0509, 142.8775, 144.2726, 145.0231, 145.8587, 146.5092, 145.9757, 146.8747, 147.7737, 148.6727, 149.5717, 150.4707, 151.3697, 152.2687, 153.1677, 154.0667, 154.9657]

men_Ha = [0.9999, 0.9999, 0.9998, 0.9999, 0.9998, 0.9997, 0.9997, 0.9997, 0.9995, 0.9994, 0.999, 0.9993, 0.9989, 0.9991, 0.9989, 0.9983, 0.9985, 0.9984, 0.998, 0.9975, 0.997, 0.9974, 0.9966, 0.9966, 0.996, 0.9953, 0.9953, 0.9951, 0.9944, 0.9945, 0.9934, 0.9936, 0.9929, 0.9931, 0.9923, 0.9925, 0.9922, 0.9913, 0.9915, 0.9911, 0.9909, 0.99, 0.9899, 0.9898, 0.9891, 0.9881, 0.9879, 0.9873, 0.9858, 0.987, 0.9855]
men_Hb = [1, 1, 0.9999, 1, 0.9999, 1, 0.9999, 1, 0.9999, 0.9998, 0.9997, 0.9999, 0.9997, 0.9998, 0.9997, 0.9995, 0.9996, 0.9995, 0.9994, 0.9994, 0.9991, 0.9993, 0.9989, 0.9988, 0.9987, 0.9986, 0.9984, 0.9982, 0.9979, 0.9977, 0.9971, 0.9972, 0.9965, 0.9962, 0.9953, 0.9948, 0.9944, 0.9929, 0.9924, 0.991, 0.9901, 0.9885, 0.986, 0.9848, 0.9835, 0.9817, 0.9798, 0.9763, 0.9748, 0.9717, 0.9662]

women_Ha = [1, 1, 1, 1, 1, 1, 1, 0.9999, 0.9999, 0.9999, 0.9998, 0.9999, 0.9998, 0.9998, 0.9998, 0.9997, 0.9997, 0.9996, 0.9996, 0.9994, 0.9993, 0.9994, 0.9992, 0.9992, 0.9991, 0.9989, 0.9989, 0.9989, 0.9986, 0.9984, 0.9981, 0.9981, 0.9982, 0.9979, 0.998, 0.9975, 0.997, 0.9969, 0.9968, 0.9964, 0.9961, 0.9956, 0.9955, 0.9951, 0.9944, 0.9939, 0.9934, 0.993, 0.99304, 0.9913, 0.9921]
women_Hb = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.9999, 0.9999, 0.9999, 0.9999, 0.9999, 0.9998, 0.9998, 0.9997, 0.9997, 0.9996, 0.9995, 0.9995, 0.9994, 0.9992, 0.9992, 0.9991, 0.9989, 0.9988, 0.9986, 0.9985, 0.9982, 0.9978, 0.9976, 0.9971, 0.9967, 0.9963, 0.9957, 0.995, 0.9941, 0.993, 0.992, 0.9911, 0.9902, 0.9885, 0.9874, 0.985, 0.9832, 0.9821, 0.9797]

Ha_list = [women_Ha, men_Ha]
Hb_list = [women_Hb, men_Hb]

mean_LDL_list = [women_mean_LDL, men_mean_LDL]
mean_SBP_list = [women_mean_SBP, men_mean_SBP]

Ha_list = [women_Ha, men_Ha]
Hb_list = [women_Hb, men_Hb]

mean_LDL_list = [women_mean_LDL, men_mean_LDL]
mean_SBP_list = [women_mean_SBP, men_mean_SBP]

# average BMI values for men and women
avg_bmi = [26.98912, 27.85051]
# average HDL values for men and women
avg_hdl = [1.598386, 1.283341]

def calculate(age, sex, ldl, ldl_rx, ldl_dec,
              age_start_rx_ldl, age_stop_rx_ldl, hdl, sbp, sbp_rx, sbp_dec,
              age_start_rx_sbp, age_stop_rx_sbp, smoke, fmr_tob, prevalent_diabetes_35,
              bmi, fam_hx_chd):

    past_a_sums = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    dm = int(prevalent_diabetes_35)

    lnRR_a_list = []
    lnRR_b_list = []

    lnRR_a = [0 for _ in range(81 - 30)]
    lnRR_b = [0 for _ in range(81 - 30)]
    a_t = [0 for _ in range(81 - 30)]
    b_t = [0 for _ in range(81 - 30)]
    c_t = [0 for _ in range(81 - 30)]
    d_t = [0 for _ in range(81 - 30)]
    e_t = [0 for _ in range(81 - 30)]
    f_t = [0 for _ in range(81 - 30)]
    m_t = [0 for _ in range(81 - 30)]

    LDL_ES_mmol = [0.610294236200972 / 45, 0.696666480552631 / 40]
    SBP_ES_mmol = [0.596822386208318 / 450, 0.562215263512178 / 400]

    del_LDL = ldl - mean_LDL_list[sex][age-30]
    del_SBP = sbp - mean_SBP_list[sex][age-30]

    a_beta = [
        [
            0.66 * 0.85, 0.5,
            [0.983679437374588, 0.768813402297644],
            [0.467034851783647, 0.417294201034895],
            0.800648203696211, 0.502003018790893,
            0.00895775920895248, -0.632265086010107,
            0.00305563
        ],
        [
            0.66 * 0.85, 0.5,
            [0.641703348526382, 0.4042777367951],
            [0.396454635449883, 0.351011034242294],
            0.690752315157672, 0.555861893739954,
            0.0109834603573848, -0.612663093385597,
            0.00305563
        ]
    ]

    b_beta = [
        [
            0.00432149552442511, 0.0000196190161699591,
            [1.11270781170849, 1.23876285117286],
            [0.336045717103265, 0.462419315914295],
            0.515257189855812, 0.0179926569543217,
            0.0140518092696023, -0.201163272313767
        ],
        [
            0.00866812947141468, 0.000260041603209642,
            [1.29162125334208, 0.857058215628963],
            [0.453034845319801, 0.382734719097288],
            0.592360632788394, 0.0374768858776153,
            0.0148344238334535, 0.00425393915728762
        ]
    ]

    for i_a in range(30, 81):
        a_sums = []
        a0 = del_LDL * a_beta[sex][0] * (i_a - 20)
        a1 = -0.12 if ldl_rx == 1 and i_a >= age_start_rx_ldl else 0
        a2 = (i_a - age_start_rx_ldl + 1) * ldl_dec if i_a >= age_start_rx_ldl and age_stop_rx_ldl - i_a >= 0 and ldl_rx == 1 else past_a_sums[-1][2]
        a3 = a0 + a2
        a4 = del_SBP * a_beta[sex][1] * (i_a - 20)
        a5 = -0.1 if sbp_rx == 1 and i_a >= age_start_rx_sbp else 0
        a6 = (i_a - age_start_rx_sbp + 1) * sbp_dec if i_a >= age_start_rx_sbp and age_stop_rx_sbp - i_a >= 0 and sbp_rx == 1 else past_a_sums[-1][6]
        a7 = a4 + a6
        a8 = LDL_ES_mmol[sex] * a3 + a1
        a9 = SBP_ES_mmol[sex] * a7 + a5

        a_sums.extend([a0, a1, a2, a3, a4, a5, a6, a7, a8, a9])
        a_sums.extend([
            smoke * a_beta[sex][2][dm],
            fmr_tob * a_beta[sex][3][dm],
            prevalent_diabetes_35 * a_beta[sex][4],
            fam_hx_chd * a_beta[sex][5],
            (bmi - avg_bmi[sex]) * a_beta[sex][6],
            (hdl - avg_hdl[sex]) * a_beta[sex][7],
        ])

        past_a_sums.append(a_sums)

        b_sums = [
            del_LDL * b_beta[sex][0],
            del_SBP * b_beta[sex][1],
            smoke * b_beta[sex][2][dm],
            fmr_tob * b_beta[sex][3][dm],
            prevalent_diabetes_35 * b_beta[sex][4],
            fam_hx_chd * b_beta[sex][5],
            (bmi - avg_bmi[sex]) * b_beta[sex][6],
            (hdl - avg_hdl[sex]) * b_beta[sex][7]
        ]

        lnRR_a_list.append(sum(a_sums[8:]))
        lnRR_b_list.append(sum(b_sums))

    for i in range(81 - 30):
        lnRR_a[i] = lnRR_a_list[i]
        lnRR_b[i] = lnRR_b_list[i]

        if i < age - 30:
            a_t[i] = 0
            b_t[i] = 0
            c_t[i] = 0
            d_t[i] = 0
            e_t[i] = 0
            f_t[i] = 0
            m_t[i] = 0

        if i == age - 30:
            a_t[i] = 1 - (Ha_list[sex][i] ** math.exp(lnRR_a[i]))
            b_t[i] = 1 - (Hb_list[sex][i] ** math.exp(lnRR_b[i]))
            c_t[i] = b_t[i]
            d_t[i] = a_t[i]
            e_t[i] = 1 - c_t[i] - d_t[i]
            f_t[i] = 0 + d_t[i]
            m_t[i] = 0 + c_t[i]
        else:
            a_t[i] = 1 - (Ha_list[sex][i] ** math.exp(lnRR_a[i]))
            b_t[i] = 1 - (Hb_list[sex][i] ** math.exp(lnRR_b[i]))
            c_t[i] = e_t[i - 1] * b_t[i]
            d_t[i] = e_t[i - 1] * a_t[i]
            e_t[i] = e_t[i - 1] - c_t[i] - d_t[i]
            f_t[i] = f_t[i - 1] + d_t[i]
            m_t[i] = m_t[i - 1] + c_t[i]

    return f_t

In [ ]:
def format_treatment(new_risk, treatments):
    global user_risk
    actions = f"lower your LDL by {treatments[0]} mmol/L and lower your SBP by {treatments[1]} mmHg".replace(' and lower your SBP by 0 mmHg', '').replace('lower your LDL by 0 mmol/L and l', 'l')
    return f"If you {actions}, then based on the risk estimation algorithm you will reduce your risk from {round(user_risk, 1)}% down to {round(new_risk * 100, 1)}%."

In [ ]:
def get_treatment(sex, age, LDL, HDL, SBP, diab, smoke, fmrtob, famhx, bmi, ldl_rx = 0, ldl_dec = 0, sbp_rx = 0, sbp_dec = 0):
    result = calculate(age, sex,
                       LDL, ldl_rx, ldl_dec, age, 80,
                       HDL,
                       SBP, sbp_rx, sbp_dec, age, 80,
                       smoke, fmrtob, diab, bmi, famhx)
    return {
        "text": format_treatment(result[-1], [abs(ldl_dec), abs(sbp_dec)]),
        "line": [age] + result
    }

In [ ]:
def within_tolerance(num1, num2, tolerance):
    return abs(num1 - num2) <= tolerance

In [ ]:
def format_recommendation(goal, treatments):
    global user_risk
    actions = f"lower your LDL by {abs(treatments[0])} mmol/L and lower your SBP by {abs(treatments[1])} mmHg".replace(' and lower your SBP by 0 mmHg', '').replace('lower your LDL by 0 mmol/L and l', 'l')
    return f"Based on the risk estimation algorithm, to get your risk down from {round(user_risk, 1)}% to {goal}% or close to it, you should aim to {actions}. However, this is just one possible course of action and you should speak to your doctor about what is healthy for you to do."

In [ ]:
def get_recommendation(sex, age, LDL, HDL, SBP, diab, smoke, fmrtob, famhx, bmi, goal):
    values = []

    for n_sbp_dec in range(0, 99-math.floor(SBP), -1):
        for n_ldl_dec in range(0, -1-math.floor(LDL), -1):
            ldl_rx = 0 if n_ldl_dec == 0 else 1
            sbp_rx = 0 if n_sbp_dec == 0 else 1

            r = calculate(age, sex,
                          LDL, ldl_rx, n_ldl_dec, age, 80,
                          HDL,
                          SBP, sbp_rx, n_sbp_dec, age, 80,
                          smoke, fmrtob, diab, bmi, famhx)

            values.append([n_ldl_dec, n_sbp_dec, r])

    pruned_paths = [e for e in values if within_tolerance(e[-1][-1]*100, goal, 0.4)]

    if len(pruned_paths) % 2 == 0:
        selected_path = pruned_paths[len(pruned_paths)//2 - 1]
    else:
        selected_path = pruned_paths[len(pruned_paths)//2]

    return {
        "text": format_recommendation(goal, selected_path[:-1]),
        "line": selected_path[-1]
    }

### Quantitative Agent Functionality

In [ ]:
actions = {
    "get_treatment": get_treatment,
    "get_recommendation": get_recommendation
}

In [ ]:
'''
Taking inspiration from agents, this branch tries to teach the vrLLM to extract
relevant pieces of the query and format it into a JSON object to pass into the
calculator.
'''

agent_template_start = '''<s>[INST] <<SYS>>
Assistant is an expert JSON builder designed to assist with a very specific task with a specific output format.

Assistant is able to trigger actions for User by only responding with JSON strings that contain "action" and "action_input" parameters.

The only actions available to Assistant are:

- "get_treatment": For when Assistant is asked how changing the User's LDL and SBP values would affect their cardiovascular risk.
  - To use the get_treatment tool, Assistant should respond with nothing but the JSON object containing "action" and "action_input", like so:
    {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": 1.5, "sbp_rx": 0, "sbp_dec": 0}}
- "get_recommendation": For when Assistant is asked to suggest what steps to take to get the User's cardiovascular risk down to a specific level.
  - To use the get_recommendation tool, Assistant should respond with nothing but the JSON object containing "action" and "action_input", like so:
    {"action": "get_recommendation", "action_input": {"goal": 5.0}}

Here are some previous conversations between the Assistant and User, where the User's current profile is included in the query:
User: What would my risk look like if I lowered my LDL by 1.5?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -1.5, "sbp_rx": 0, "sbp_dec": 0}}
User: What would my risk look like if I decrease my LDL to 1.5?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": 1.5 - ldl, "sbp_rx": 0, "sbp_dec": 0}}
User: How much would my risk lower if I reduced my blood pressure to 120?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 0, "ldl_dec": 0, "sbp_rx": 1, "sbp_dec": 120 - sbp}}
User: What would my risk look like if I lowered my SBP by 20 points?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 0, "ldl_dec": 0, "sbp_rx": 1, "sbp_dec": -20}}
User: What would my risk look like if I decrease my SBP by 30?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 0, "ldl_dec": 0, "sbp_rx": 1, "sbp_dec": -30}}
User: What would my risk look like if I decrease my LDL by 2.0?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -2.0, "sbp_rx": 0, "sbp_dec": 0}}
User: What would my risk look like if I cut my LDL in half?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -ldl/2, "sbp_rx": 0, "sbp_dec": 0}}
User: If my blood pressure decreases to 140, how much would that affect my risk?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 0, "ldl_dec": 0, "sbp_rx": 1, "sbp_dec": 140 - sbp}}
User: If I reduce my LDL to 3.5 mmol/L, what would my risk look like?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": 3.5 - ldl, "sbp_rx": 0, "sbp_dec": 0}}
User: If I reduce my LDL by 1.5 points, what would my risk look like?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -1.5, "sbp_rx": 0, "sbp_dec": 0}}
User: What would my risk look like if I lowered my LDL by 1.5 and my SBP by 20 points?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -1.5, "sbp_rx": 1, "sbp_dec": -20}}
User: What would my risk look like if I decrease my SBP by 30 and my LDL by 0.5?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 1, "ldl_dec": -0.5, "sbp_rx": 1, "sbp_dec": -30}}
User: If I took medication to lower my blood pressure by 10 points, how would that improve by risk score?
Assistant: {"action": "get_treatment", "action_input": {"ldl_rx": 0, "ldl_dec": 0, "sbp_rx": 1, "sbp_dec": -10}}
User: What should I do to lower my risk to 5%?
Assistant: {"action": "get_recommendation", "action_input": {"goal": 5.0}}
User: How do I reduce my risk by half?
Assistant: {"action": "get_recommendation", "action_input": {"goal": goal/2}}
User: How much would I have to lower my cholesterol or blood pressure to reduce my risk by 2.0%?
Assistant: {"action": "get_recommendation", "action_input": {"goal": goal - 2.0}}
User: How do I cut my risk in half?
Assistant: {"action": "get_recommendation", "action_input": {"goal": goal/2}}

<</SYS>>

'''
agent_template_end = '''
[/INST]
Assistant:
'''

In [ ]:
def process_command(command):
    # Format the prompt with the user's command
    prompt = agent_template_start + "User: " + command + agent_template_end

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors = "pt").to(device)

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens = 200,
    )
    response_text = tokenizer.decode(outputs[0], skip_special_tokens = True)

    response_text = response_text.split('User:')[-1]

    # Try to find JSON in the response - sometimes it outputs more than needed
    json_match = re.findall(r'\{.*\}', response_text)

    return json_match[0]

In [ ]:
def serve_quantitative(prompt):
    result = json.loads(process_command(prompt))

    global user_profile
    global user_risk

    # Some of the possible patterns are based on the current LDL, SBP, or risk.
    ldl = user_profile['LDL']
    sbp = user_profile['SBP']
    goal = user_risk
    treatment_line = actions[result["action"]](**user_profile, **result["action_input"])

    return treatment_line

## Self-Reflection

The self-reflection algorithm is also given access to the CAKE.

In [ ]:
reflection_system_prompt = """[INST] <<SYS>>
You are an evaluator designed to assist with a very specific task and respond in a very specific way.
You will receive a Prompt and a Provisional Response as an input.

You will evaluate the provisional response based on the given prompt.
You will determine if the response:
1. Accurately reflects the details provided in the prompt.
2. Is free from discrepancies or contradictions with the prompt.
3. Does not introduce any new information not present in the prompt.
4. Does not directly reference the "provided context" or use similar phrases.
5. Is not too short and includes all important details.

If the response meets all of these criteria satisfactorily, you should return the original provisional response unchanged.
If the response fails to meet one or more of these criteria, you should generate an improved response using only the information contained in the prompt, or modify the existing reponse.

Your output should ONLY be one of the following:
1. The original provisional response unchanged.
2. The improved response without any preamble, explanations, or additional text.
<</SYS>>
"""

reflection_query_wrapper_prompt = SimpleInputPrompt("{query_str}[/INST]")

# Create a HF LLM using the llama index wrapper
reflection_llm = HuggingFaceLLM(
    context_window = 4096,
    max_new_tokens = 512,
    generate_kwargs = {
        "temperature": 0.5,
        "top_p": 0.9,
        "top_k": 50,
    },
    system_prompt = reflection_system_prompt,
    query_wrapper_prompt = reflection_query_wrapper_prompt,
    model = model,
    tokenizer = tokenizer
)

def self_reflect(prompt, response):
  # Setup index query engine using LLM
  query_engine = doc_index.as_query_engine(llm = reflection_llm)
  prompt = f"Prompt: {prompt}\nProvisional Response: {response}"
  verdict = query_engine.query(prompt)

  return verdict.response.strip()

# User profile


```
Global variable
```

In [ ]:
def update_matching_fields(target_dict, source_dict):
    for key in source_dict:
        if key in target_dict:
            target_dict[key] = source_dict[key]

In [ ]:
'''
One field per relevant input box in the profile. Height and weight not included
due to BMI being the relevant field for the calculation algorithm.
get_treatment and get_recommendation receive additional fields so it's not necessary
for user_profile to include things like "ldl_rx" etc.
'''

user_profile = {
    'sex': None,
    'age': None,
    'LDL': None,
    'HDL': None,
    'SBP': None,
    'diab': None,
    'smoke': None,
    'fmrtob': None,
    'famhx': None,
    'bmi': None
}

In [ ]:
user_risk = None

# Main Stream

In [ ]:
# Initialize the Flask app and the context history
app = Flask("expert-bot", static_folder = STATIC_DIR)
context_history = []

In [ ]:
def initialize_context():
    # Define an initial conversation context.
    # The system prompt attempts to counteract the friendliness of the model.
    initial_context = [
        "System: Please answer the questions put to you without emotion or greeting. Don't mention the context in your answers either."
    ]

    return initial_context

In [ ]:
@app.route("/")
def home():
    # Location of the HTML document for the frontend
    html_file_path = '/content/interface/index.html'
    with open(html_file_path, 'r') as file:
        html_content = file.read()

    return html_content

# Calculate Interaction

In [ ]:
'''
This is the initial calculation, that the user must complete to interact with
the bot logic. This ensures that there is a copy of the user's profile in the
active variables so that get_treatment and get_recommendation can use it.
'''

@app.route("/calculate", methods=["POST"])
def calculate_risk():
    profile_state = request.get_json()
    profile_state = {key: float(value) if '.' in value else int(value) for key, value in profile_state.items()}

    global user_profile
    update_matching_fields(user_profile, profile_state)

    age = profile_state['age']
    sex = profile_state['sex']
    ldl = profile_state['LDL']
    ldl_rx = profile_state['ldl_rx']
    ldl_dec = profile_state['ldl_dec']
    age_start_rx_ldl = age
    age_stop_rx_ldl = 80

    sbp = profile_state['SBP']
    sbp_rx = profile_state['sbp_rx']
    sbp_dec = profile_state['sbp_dec']
    age_start_rx_sbp = age
    age_stop_rx_sbp = 80

    hdl = profile_state['HDL']
    bmi = profile_state['bmi']
    diab = profile_state['diab']
    smoke = profile_state['smoke']
    fmr_tob = profile_state['fmrtob']
    famhx = profile_state['famhx']

    result = calculate(age, sex,
                       ldl, ldl_rx, ldl_dec, age_start_rx_ldl, age_stop_rx_ldl,
                       hdl,
                       sbp, sbp_rx, sbp_dec, age_start_rx_sbp, age_stop_rx_sbp,
                       smoke, fmr_tob, diab, bmi, famhx)

    output = {}
    output['age'] = age
    output['data'] = result

    global user_risk
    user_risk = result[-1] * 100

    return jsonify(output)

# Hub Interaction

In [ ]:
'''
As a personal preference, I don't like that LLMs have this artefact despite the
fine-tuning. I therefore get rid of it if it shows up.
'''

pattern = r"^Sure,?.*?that[.!]?\s" # Starts with "Sure" and matches up to the first exclamation mark or fullstop.

In [ ]:
@app.route("/interact", methods=["POST"])
def interact():
    free_gpu_memory()
    global context_history
    data = request.get_json()
    user_input = data['query']

    branch = classify(stage1_classifier, classifier_tokenizer, user_input, label_dict)
    print(f'branch: {branch}')

    if branch == 'general information':
      # Append user input to context as needed
      context_history.append(f"User: {user_input}")

      # Knowledge-based questions keep a context for the conversation.
      conversation = '\n'.join(context_history)
      response_text = serve_knowledge(conversation)

      # Append the formatted response to the context
      context_history.append(f"Assistant: {response_text['text']}")
      response_text['text'] = re.sub(pattern, "", response_text['text'].replace('Sure!', 'Sure,'), flags=re.DOTALL).strip()

      # Maintain a recent context window to avoid stale conversation artifacts
      if len(context_history) > 9:
          # keep the last 9 exchanges (4 User/Assistant pairs and the System prompt)
          context_history = context_history[0] + context_history[-8:]

      # Self reflection
      response_text['text'] = self_reflect(user_input, response_text['text'])

      return jsonify(response_text)

    elif branch == 'quantitative analysis':
        response_text = serve_quantitative(user_input)
        return jsonify(response_text)

    else:
        # If an irrelevant question is asked it never reaches the vrLLM.
        response_text = {"text": "Sorry, I'm not able to help you with that. Please either rephrase the question or ask a different question.", "line": None}
        return jsonify(response_text)

In [ ]:
if __name__ == '__main__':
    public_url = ngrok.connect(7000)

    print(f"Flask app is running at {public_url}")

    # Run the Flask app
    app.run(host='0.0.0.0', port=7000)